# Init

Install Libraries

In [ ]:
!pip install torch transformers faiss-cpu langchain PyPDF2 pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.5/42.5 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 41.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Mount Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# from transformers import AutoTokenizer, AutoModel
# import torch

# # Load SciBERT tokenizer & model
# model_name = "allenai/scibert_scivocab_uncased"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)

# # Function to get embeddings
# def get_embedding(text):
#     inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
#     with torch.no_grad():
#         outputs = model(**inputs)
#     return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

Initialize FAISS

In [ ]:
# import faiss
# import numpy as np

# # Initialize FAISS index
# d = 512  # T5 output dimension
# index = faiss.IndexFlatIP(d)

In [ ]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


#Extraction

1.   Extract Text
2.   Convert to chunks
3.   Pickle for later use



Extract Text

In [ ]:
# import textwrap

# def chunk_text(text, chunk_size=500):
#     return textwrap.wrap(text, width=chunk_size)

In [ ]:
#path to the papers
pdf_directory = "/content/drive/Shareddrives/ECS_289G/papers"

In [ ]:
# import pdfplumber
# from transformers import BertTokenizer, BertModel

# # Initialize SciBERT tokenizer and model
# tokenizer = BertTokenizer.from_pretrained('allenai/scibert_scivocab_uncased')
# model = BertModel.from_pretrained('allenai/scibert_scivocab_uncased').to(device)

# # Function to extract text from PDFs
# def extract_text_from_pdf(pdf_path):
#     with open(pdf_path, "rb") as file:
#         reader = PyPDF2.PdfReader(file)
#         text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])

#     sentences = text.split('.')
#     return sentences

In [ ]:
def sliding_window_chunking_raw(text, window_size=2048, overlap=512):
    """
    Chunk the raw text using the sliding window approach.
    """
    chunks = []
    for i in range(0, len(text), window_size - overlap):
        chunk = text[i:i + window_size]
        chunks.append(chunk)

        # Stop if we've reached the end of the text
        if i + window_size >= len(text):
            break

    return chunks


In [ ]:
import pdfplumber

# Function to extract text from PDFs
def extract_text_from_pdf(pdf_path):
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        text = "\n".join([page.extract_text() for page in reader.pages if page.extract_text()])

    chunks = sliding_window_chunking_raw(text)
    return chunks

In [ ]:
from transformers import T5Tokenizer, T5Model
tokenizer = T5Tokenizer.from_pretrained('t5-small')  # or 't5-large'
model = T5Model.from_pretrained('t5-small').to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [ ]:
# s = "I like dancing. Dancing is my hobby"
# print(s.split('.'))

In [ ]:
# Function to generate embedding for text using SciBERT
def get_embedding(text):
    if not text or not isinstance(text, str):
        raise ValueError("Input text must be a non-empty string")

    inputs = tokenizer(text, return_tensors='pt', truncation=True, padding=True, max_length=4096).to(device)

    with torch.no_grad():
        outputs = model.encoder(**inputs)  # Use the encoder to get embeddings

    embedding = outputs.last_hidden_state.mean(dim=1).squeeze().cpu().numpy().reshape(1,-1).astype('float32')
    return embedding

In [ ]:
import numpy as np

def normalize_embeddings(embeddings):
    norm = np.linalg.norm(embeddings, axis=1, keepdims=True)
    return embeddings / norm

In [ ]:
# chunk_map = {}

In [ ]:
# import os
# import PyPDF2

# chunk_id = 0  # ID for the text chunks
# for filename in os.listdir(pdf_directory):
#     if filename.endswith(".pdf"):
#         pdf_path = os.path.join(pdf_directory, filename)
#         print(f"Processing {filename}...")

#         # Extract text from the PDF
#         sentences = extract_text_from_pdf(pdf_path)

#         # Split text into smaller chunks if needed (e.g., paragraphs or sentences)
#         # text_chunks = chunk_text(text)  # You can split by sentence or paragraphs

#         # Generate embeddings for each chunk
#         for sentence in sentences:
#               embedding = get_embedding(sentence)
#               embedding = embedding.reshape(1, -1)  # Reshape to (1, d)
#               normalize_embedding = normalize_embeddings(embedding)
#               # Add the embedding to FAISS index
#               index.add(normalize_embedding)


#               # Store the chunk with a unique ID
#               chunk_map[chunk_id] = (sentence, filename)
#               chunk_id += 1

Processing A novel classification method for paper-reviewer recommendation.pdf...
Processing A Novel Method for Video Tracking Performance Evaluation.pdf...
Processing A_Novel_Method_of_Combined_Feature_Extraction_for_Recognition.pdf...
Processing A_Novel_Method_for_Detecting_Cropped_and_Recompressed_Image_Block.pdf...
Processing Analysis of Practicality and Performance Evaluation for Monolithic Kernel and Micro-Kernel Operating Systems.pdf...
Processing A Novel Approach for E-mail Classification Using FastText.pdf...
Processing A_novel_method_for_tracking_and_counting_pedestrians_in_real-time_using_a_single_camera.pdf...
Processing A novel method to look for the hysteresis thresholds for the Canny Edge Detector.pdf...
Processing A Case Against CXL Memory Pooling.pdf...
Processing A_novel_method_to_detect_Heart_Beat_Rate_using_a_mobile_phone.pdf...
Processing Camouflage_Defect_Identification_A_Novel_Approach.pdf...
Processing Gender Classification using Facial Embeddings A Novel Approa

In [1]:
print(len(chunk_map))
print(index.ntotal)

In [ ]:
import pickle

# Save the FAISS index and chunk mapping to Google Drive
faiss.write_index(index, "/content/drive/Shareddrives/ECS_289G/faiss_index_1.idx")
with open("/content/drive/Shareddrives/ECS_289G/chunk_map_1.pkl", "wb") as f:
    pickle.dump(chunk_map, f)

print("FAISS index and chunk mapping saved successfully!")

FAISS index and chunk mapping saved successfully!


In [ ]:
# #embeddings
# from transformers import AutoTokenizer, AutoModel
# import torch

# # Load SciBERT tokenizer & model
# model_name = "allenai/scibert_scivocab_uncased"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)

# # Function to get embeddings
# def get_embedding(text):
#     inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
#     with torch.no_grad():
#         outputs = model(**inputs)
#     return outputs.last_hidden_state.mean(dim=1).squeeze().numpy()


In [ ]:
pwd

'/content'

In [ ]:
import faiss

index = faiss.read_index("/content/drive/Shareddrives/ECS_289G/faiss_index.idx")

In [ ]:
import pickle

# Specify the path to your pickle file
pickle_file_path = '/content/drive/Shareddrives/ECS_289G/chunk_map.pkl'

# Open the pickle file in binary read mode
with open(pickle_file_path, 'rb') as f:
    chunk_map = pickle.load(f)

# Verify the extracted dictionary
print(chunk_map)

{0: ('A novel classiﬁcation method for paper-reviewer\nrecommendation\nShu Zhao1•Dong Zhang1•Zhen Duan1•Jie Chen1•Yan-ping Zhang1•\nJie Tang2\nReceived: 3 November 2017 / Published online: 31 March 2018\n/C211Akade ´miai Kiado ´, Budapest, Hungary 2018\nAbstract Reviewer recommendation problem in the research ﬁeld usually refers to invite\nexperts to comment on the quality of papers, proposals, etc. How to effectively and\naccurately recommend reviewers for the submitted papers and proposals is a meaningful\nand still tough task. At present, many unsupervised recommendation methods have been\nresearched to solve this task. In this paper, a novel classiﬁcation method named Word\nMover’s Distance–Constructive Covering Algorithm (WMD–CCA, for short) is proposed\nto solve the reviewer recommendation problem as a classiﬁcation issue. A submission or a\nreviewer is described by some tags, such as keywords, research interests, and so on. First,\nthe submission or the reviewer is represented a

In [ ]:
# Check if the index is trained (for trained indices)
print("Is the index trained?", index.is_trained)

# Print the index type
print("Index type:", type(index))

# Check the number of vectors in the index
print(f"Number of vectors in the index: {index.ntotal}")

Is the index trained? True
Index type: <class 'faiss.swigfaiss_avx512.IndexFlatIP'>
Number of vectors in the index: 1826


In [ ]:
import numpy as np

def search_faiss(query, top_k=15):
    query = get_embedding(query)
    query_embedding = np.array([query])
    embedding = query_embedding.reshape(1, -1)
    normalize_embedding = normalize_embeddings(embedding)
    distances, indices = index.search(normalize_embedding, top_k)
    return [chunk_map[idx] for idx in indices[0]]

In [ ]:
def create_prompt(query, retrieved_limitations, retrieved_previous_implementations, current_implementation):
    prompt = f"""
You are an AI assistant evaluating the feasibility and novelty of a research approach based on retrieved information. Only focus on the relevant limitations and implementations related to the query. Strictly ignore any unrelated content.

**Query:**
{query}

**Current Implementation:**
{current_implementation}

**Retrieved Limitations (For Feasibility Analysis):**
{retrieved_limitations}

**Retrieved Previous Implementations (For Novelty Analysis):**
{retrieved_previous_implementations}

**Evaluation Criteria:**

Feasibility Rubric:
1. Highly Impractical (Rating 1)

Challenges: The approach faces fundamental flaws, such as severe technical limitations, critical errors in assumptions, or unsolvable scalability issues.
Implementation: The idea cannot be realistically executed with existing tools or methods and would fail under typical conditions.
Risk: There are significant risks (e.g., security, performance) that cannot be easily mitigated or are likely to lead to failure.
Examples: Security issues that cannot be resolved, approaches that are computationally unfeasible at scale, or major technical hurdles without clear solutions.
2. Challenging to Implement (Rating 2)

Challenges: The idea presents significant obstacles, such as major performance bottlenecks, security risks, or issues with scalability that would make it difficult to execute effectively.
Implementation: The idea requires substantial adjustments or resources to become viable, and careful planning is necessary for success.
Risk: High risk, including issues that could lead to significant failure unless properly managed.
Examples: Major data bottlenecks, scalability challenges, or risks of model instability (like hallucinations) that require significant innovation or effort to mitigate.
3. Feasible with Significant Effort (Rating 3)

Challenges: The approach is achievable but faces clear challenges that need to be addressed, such as optimization, error handling, or complex dependencies.
Implementation: The idea can be executed, but substantial resources, testing, and refinement are required.
Risk: Moderate risk, where careful management of the process is needed to avoid potential pitfalls (e.g., handling edge cases, scalability).
Examples: A viable idea but requires significant engineering work, model fine-tuning, and optimization to be effective and scalable.
4. Mostly Feasible with Some Risk (Rating 4)

Challenges: The approach works well in most conditions with only minor risks or challenges that can be addressed through optimization or incremental improvements.
Implementation: Practical to implement with manageable risks, requiring some refinement but no major adjustments to current resources.
Risk: Low risk, where the approach is likely to succeed but with some areas needing attention (e.g., improving model stability).
Examples: A solution that can be scaled with moderate adjustments or where only minor technical challenges remain.
5. Highly Feasible with Minimal or No Risk (Rating 5)

Challenges: The approach is fully practical, and any minor limitations can be easily addressed.
Implementation: Ready to be implemented without major risks or limitations.
Risk: Very low risk, and the approach is efficient, scalable, and well-suited to current methods.
Examples: A proven method with minimal refinement needed, able to handle expected scenarios smoothly.
Novelty Rubric:
Novelty Rubric:

Identical to Existing Methods (Rating 1)

Contribution: The approach is nearly identical to prior implementations without any meaningful change or adaptation. It replicates existing work with no significant variation.
Innovation: The idea offers no new insights or improvements over previous implementations.
Examples: Direct repetition of previously applied techniques with no unique adaptations or variations.

Minor Variations but Largely Follows Prior Work (Rating 2)

Contribution: The approach introduces minor changes to previous implementations but does not make any significant contribution to the field.
Innovation: There is some degree of originality, but the method or solution still aligns closely with previous work.
Examples: Slight modifications to methods or frameworks that overlap heavily with existing solutions without adding substantial value.

Some Novelty but Overlaps Significantly (Rating 3)

Contribution: The idea introduces some novel elements or improvements compared to previous implementations, but the core concept or methodology remains similar to existing work.
Innovation: It presents meaningful novelty but builds on the foundation of prior implementations.
Examples: New applications or improvements that enhance the current method but don't fully break from previous models or implementations.

Distinct Contributions (Rating 4)

Contribution: The approach offers a significant advancement over prior implementations, incorporating new ideas, methods, or techniques that meaningfully differ from the existing work.
Innovation: The idea is a clear departure from prior work, contributing new insights, frameworks, or strategies that advance the field.
Examples: Introduction of new methodologies, algorithms, or frameworks that noticeably improve or expand upon existing solutions.

Highly Novel and Groundbreaking (Rating 5)

Contribution: The idea presents a groundbreaking shift or entirely new methodology that significantly deviates from previous work, leading to a major innovation in the field.
Innovation: The approach introduces a disruptive concept that changes the understanding of the problem or solution, with far-reaching implications.
Examples: A revolutionary idea that redefines current approaches or introduces a new paradigm, leading to major changes in the field.

Feasible: Approaches rated 4-5.
Infeasible: Approaches rated 1-3.
Novelty Classification:

Novel: Approaches rated 4-5.
Not Novel: Approaches rated 1-3.
"""
    return prompt


In [ ]:
# user_input = input()


In [ ]:
implementation = input()

Camouflaged object detection (COD) aims to detect/segment camouflaged objects embedded in the environment, which has attracted increasing attention over the past decades. Although several COD methods have been developed, they still suffer from unsatisfactory performance due to the intrinsic similarities between the foreground objects and background surroundings. In this paper, we propose a novel Feature Aggregation and Propagation Network (FAP-Net) for camouflaged object detection. Specifically, we propose a Boundary Guidance Module (BGM) to explicitly model the boundary characteristic, which can provide boundary-enhanced features to boost the COD performance. To capture the scale variations of the camouflaged objects, we propose a Multi-scale Feature Aggregation Module (MFAM) to characterize the multi-scale information from each layer and obtain the aggregated feature representations. Furthermore, we propose a Cross-level Fusion and Propagation Module (CFPM). In the CFPM, the feature 

In [ ]:
# query = "Given abstract and proposed solution " + user_input + "What are the key performance, scalability, and cost benefits of the proposed system compared to traditional approaches in [domain]? How does the new system address previous limitations and what new challenges or trade-offs does it introduce? What are the real-world implications and practical limitations of implementing this system, and how does its performance hold up under various conditions (e.g., workloads, resource allocation)"

In [ ]:
# query = ''' What are the sources, formats, and characteristics of datasets commonly used in ''' + topic

In [ ]:
# query='''What are the real-world constraints and practical challenges in deploying Proposed Solution at scale?'''+topic

In [ ]:
limitation_query = f"What are the limitations of {implementation}?"

In [ ]:
print(limitation_query)

What are the limitations of Camouflaged object detection (COD) aims to detect/segment camouflaged objects embedded in the environment, which has attracted increasing attention over the past decades. Although several COD methods have been developed, they still suffer from unsatisfactory performance due to the intrinsic similarities between the foreground objects and background surroundings. In this paper, we propose a novel Feature Aggregation and Propagation Network (FAP-Net) for camouflaged object detection. Specifically, we propose a Boundary Guidance Module (BGM) to explicitly model the boundary characteristic, which can provide boundary-enhanced features to boost the COD performance. To capture the scale variations of the camouflaged objects, we propose a Multi-scale Feature Aggregation Module (MFAM) to characterize the multi-scale information from each layer and obtain the aggregated feature representations. Furthermore, we propose a Cross-level Fusion and Propagation Module (CFPM

In [ ]:
print(implementation)

Camouflaged object detection (COD) aims to detect/segment camouflaged objects embedded in the environment, which has attracted increasing attention over the past decades. Although several COD methods have been developed, they still suffer from unsatisfactory performance due to the intrinsic similarities between the foreground objects and background surroundings. In this paper, we propose a novel Feature Aggregation and Propagation Network (FAP-Net) for camouflaged object detection. Specifically, we propose a Boundary Guidance Module (BGM) to explicitly model the boundary characteristic, which can provide boundary-enhanced features to boost the COD performance. To capture the scale variations of the camouflaged objects, we propose a Multi-scale Feature Aggregation Module (MFAM) to characterize the multi-scale information from each layer and obtain the aggregated feature representations. Furthermore, we propose a Cross-level Fusion and Propagation Module (CFPM). In the CFPM, the feature 

In [ ]:
top_results_implementations = search_faiss(implementation, top_k=6)

retrieved_implementations = [result[0] for result in top_results_implementations]
for i, result in enumerate(top_results_implementations, 1):
    print(f"Result {i}:\nPaper - {result[1]}\n\n")

top_results_limitations = search_faiss(limitation_query, top_k=6)

retrieved_limitations = [result[0] for result in top_results_limitations]
for i, result in enumerate(top_results_limitations, 1):
    print(f"Result {i}:\nPaper - {result[1]}\n\n")

Result 1:
Paper - Adaptive Graph Convolutional Recurrent Network.pdf


Result 2:
Paper - NEUROLM A UNIVERSAL MULTI-TASK FOUNDATION.pdf


Result 3:
Paper - Addressing-failure-prediction-by-learning-model-confidence-Paper.pdf


Result 4:
Paper - CogVideo Large-scale Pretraining for Text-to-Video.pdf


Result 5:
Paper - Noise Estimation Using Density Estimation.pdf


Result 6:
Paper - The CHERI capability model- Revisiting RISC in an age of risk.pdf


Result 1:
Paper - Adaptive Graph Convolutional Recurrent Network.pdf


Result 2:
Paper - NEUROLM A UNIVERSAL MULTI-TASK FOUNDATION.pdf


Result 3:
Paper - Addressing-failure-prediction-by-learning-model-confidence-Paper.pdf


Result 4:
Paper - CogVideo Large-scale Pretraining for Text-to-Video.pdf


Result 5:
Paper - Noise Estimation Using Density Estimation.pdf


Result 6:
Paper - NEUROLM A UNIVERSAL MULTI-TASK FOUNDATION.pdf




In [ ]:
query = input()

Resource-Aware Saliency-Guided Differentiable Pruning for Deep Neural Networks


In [ ]:
prompt = create_prompt(query, retrieved_limitations, retrieved_implementations, implementation)
print(prompt)


You are an AI assistant evaluating the feasibility and novelty of a research approach based on retrieved information. Only focus on the relevant limitations and implementations related to the query. Strictly ignore any unrelated content.

**Query:**
Resource-Aware Saliency-Guided Differentiable Pruning for Deep Neural Networks

**Current Implementation:**
The increasing demand for efficient deep learning model deployment on Tiny Machine Learning (tinyML) and Edge platforms necessitates the development of methods that enable automated and effective network pruning, tailored to tinyML hardware constraints. In this paper, we present a novel differentiable pruning method that accepts total available memory on a tinyML hardware and employs saliency based measurements to identify and prune less significant connections within a deep neural network (DNN). Our approach integrates network compression within the training process, adapting resource utilization to the specific constraints of FPGAs